In [1]:
from numpy.core.fromnumeric import shape
import glob
from IPython.core.display import Image
from numpy.lib.function_base import append
import numpy as np
from google.colab.patches import cv2_imshow
import cv2 # 導入OpenCV函式庫
from google.colab import files

pts = [] #right images points
ptsp = [] #left images points

F = np.array([[-0.000000076386, 0.000003333642, -0.001745446202],
    [0.000000105635, -0.000000103805, -0.011352133262],
    [-0.000285872783, 0.010010406460, 1.000000000000]])

LK = np.array([[1035.278669095568, 0.000000000000, 295.500377771516],
    [0.000000000000, 1034.880664685675, 598.224722223280],
    [0.000000000000, 0.000000000000, 1.000000000000]])
RK = np.array([[1036.770200759934, 0.000000000000, 403.040387412710],
    [0.000000000000, 1037.186415753241, 612.775486819306],
    [0.000000000000, 0.000000000000, 1.000000000000]])
LRT = np.array([[1.000000000000, 0.000000000000, 0.000000000000, 0.000000000000],
     [0.000000000000, 1.000000000000, 0.000000000000, 0.000000000000],
     [0.000000000000, 0.000000000000, 1.000000000000, 0.000000000000]])
RRT = np.array([[0.958173249509, 0.009400631103, 0.286034354684, -69.855978076557],
     [-0.009648701145, 0.999953303246, -0.000542119475, 0.110435878514],
     [-0.286026094074, -0.002240415626, 0.958219209809, 14.517584144224]])

LP = np.dot(LK,LRT)
RP = np.dot(RK,RRT)


# ====================load  R images==================================
Rimagelist = sorted(glob.glob('/content/R/'+'R*.JPG'))
# print(Rimagelist)
Rimg_array = []
for i in range(len(Rimagelist)):
  img = cv2.imread(Rimagelist[i])
  Rimg_array.append(img)
Rimg_array = np.array(Rimg_array)
# ====================load  L images==================================
Limagelist = sorted(glob.glob('/content/L/'+'L*.JPG'))
# print(Limagelist)
Limg_array = []
for i in range(len(Limagelist)):
  img = cv2.imread(Limagelist[i])
  Limg_array.append(img)
Limg_array = np.array(Limg_array)
# ==================================================================


# ====================find corresponding points for every images==================================
for k in range(len(Rimg_array)):
  #pick brightest point of R
  Rred = []
  for i in range(len(Rimg_array[k])):
    if Rimg_array[k,i,Rimg_array[k,i,:,2].argmax(),2] >= 230:#if the R channel value of pixl is bigger then 230,pick it
      Rred.append([Rimg_array[k,i,:,2].argmax(),i])
  # print(Rred)
  #pick brightest point of L
  Lred = []
  for i in range(len(Limg_array[k])):
    if Limg_array[k,i,Limg_array[k,i,:,2].argmax(),2] >= 230:#if the R channel value of pixl is bigger then 230,pick it
      Lred.append([Limg_array[k,i,:,2].argmax(),i])
  # print(Lred)

  for p in range(len(Rred)):
    pts.append(Rred[p])

  Rred = np.array(Rred)
  Lred = np.array(Lred)
  # find epipllae line of every point ==========================================
  lines = cv2.computeCorrespondEpilines(Rred.reshape(-1,1,2),2,F)
  lines = lines.reshape(-1,3)
  # print(lines)

  for i in range(len(lines)):
    dis_err = []
    for j in range(0, len(Lred)):
      dis = abs((lines[i,0]*Lred[j,0]+lines[i,1]*Lred[j,1]+lines[i,2]))/np.sqrt(lines[i,0]**2+lines[i,1]**2)
      dis_err.append(dis)
      # print('point',j,'error:',dis)
    dis_err = np.array(dis_err)
    ptsp.append(Lred[dis_err.argmin()])

#  save corresponding points
pts = np.array(pts)
ptsp = np.array(ptsp)
# ==================================================================

# ====================find xyz coordinate===========================
P_world = []
for f in range(len(pts)):
  A = np.array([pts[f,0]*RP[2]-RP[0],
                pts[f,1]*RP[2]-RP[1],
                ptsp[f,0]*LP[2]-LP[0],
                ptsp[f,1]*LP[2]-LP[1]])

  U, S, Vt = np.linalg.svd(A)
  V = Vt.T
  P_world.append(V[:,3]/V[3,3])

P_world=np.array(P_world)
P_save = P_world[:,:3]

P_save = P_save.astype(int)

# ====================save xyz file===========================
np.savetxt('M11012002.xyz', P_save)
files.download("M11012002.xyz")


['/content/R/R000.JPG', '/content/R/R001.JPG', '/content/R/R002.JPG', '/content/R/R003.JPG', '/content/R/R004.JPG', '/content/R/R005.JPG', '/content/R/R006.JPG', '/content/R/R007.JPG', '/content/R/R008.JPG', '/content/R/R009.JPG', '/content/R/R010.JPG', '/content/R/R011.JPG', '/content/R/R012.JPG', '/content/R/R013.JPG', '/content/R/R014.JPG', '/content/R/R015.JPG', '/content/R/R016.JPG', '/content/R/R017.JPG', '/content/R/R018.JPG', '/content/R/R019.JPG', '/content/R/R020.JPG', '/content/R/R021.JPG', '/content/R/R022.JPG', '/content/R/R023.JPG', '/content/R/R024.JPG', '/content/R/R025.JPG', '/content/R/R026.JPG', '/content/R/R027.JPG', '/content/R/R028.JPG', '/content/R/R029.JPG', '/content/R/R030.JPG', '/content/R/R031.JPG', '/content/R/R032.JPG', '/content/R/R033.JPG', '/content/R/R034.JPG', '/content/R/R035.JPG', '/content/R/R036.JPG', '/content/R/R037.JPG', '/content/R/R038.JPG', '/content/R/R039.JPG', '/content/R/R040.JPG', '/content/R/R041.JPG', '/content/R/R042.JPG', '/content/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>